In [78]:
import os

In [79]:
%pwd

'C:\\Users\\jeeva\\Desktop\\MLOPS-Project'

In [80]:
os.chdir("../")

In [81]:
%pwd

'C:\\Users\\jeeva\\Desktop'

In [82]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/jeeva-eng/MLOPS-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="jeeva-eng"
os.environ["MLFLOW_TRACKING_PASSWORD"]="11f70d0aa3d72d459b18fb5f516f9df6dec7015b"

In [83]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:Path
    test_data_path:Path
    model_path:Path
    all_params:dict
    metric_file_name:Path
    target_columns:str
    mlflow_uri:str

In [84]:
from src.mlops_project.constants import*
from src.mlops_project.utils.common import read_yaml,create_directories,save_json

In [85]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMNS

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_columns=schema.name,
            mlflow_uri="https://dagshub.com/jeeva-eng/MLOPS-Project.mlflow"
        )

        return model_evaluation_config

In [86]:
import os
os.chdir("C:/Users/jeeva/Desktop/MLOPS-Project")
print(os.getcwd())

C:\Users\jeeva\Desktop\MLOPS-Project


In [87]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [88]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config

    def eval_metrics(self,actual,pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)
        return rmse,mae,r2


    def log_into_mlflow(self):

        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_columns],axis=1)
        test_y=test_data[self.config.target_columns]  


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities=model.predict(test_x)

            (rmse,mae,r2)=self.eval_metrics(test_y,predicted_qualities)

            # saving metrics as a local

            scores={"rmse":rmse,"mae":mae,"r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)



            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)

            # Model registry does not work with file store

            if tracking_url_type_store != "file":

                # register the model
                # there are other ways to use the Model Registry, which depends on the use case
                # pleases refer to doc for more information

                mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model,"model")    


In [89]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation_config=ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()

except Exception as e:
    raise e    

[2026-02-22 12:16:35,611:INFO:common:yaml file:config\config.yaml loaded successfully]
[2026-02-22 12:16:35,617:INFO:common:yaml file:params.yaml loaded successfully]
[2026-02-22 12:16:35,622:INFO:common:yaml file:schema.yaml loaded successfully]
[2026-02-22 12:16:35,624:INFO:common:Created Directory at:artifacts]
[2026-02-22 12:16:35,628:INFO:common:Created Directory at:artifacts/model_evaluation]
[2026-02-22 12:16:39,102:INFO:common:json file saved at:artifacts\model_evaluation\metrics.json]


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ElasticnetModel'.
2026/02/22 12:17:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
